In [1]:
#!pip install contextily
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import folium
import pandas as pd

In [80]:
df_pois = gpd.read_file('gis_osm_pois_a_free_1.shp')

In [81]:
"""
I Only keep leisure and some shops and markets (see documentation : https://download.geofabrik.de/osm-data-in-gis-formats-free.pdf) 
"""

df_pois  = df_pois[(df_pois["code"] >= 2201) 
                   & (df_pois["code"] <= 2706)]

In [82]:

df_pois = df_pois.reset_index(drop = True) #I restart index  

In [83]:
df_pois 

,osm_id,code,fclass,name,geometry
0,4083189,2204,park,Parc de Bercy,"POLYGON ((2.37807 48.83739, 2.37812 48.83741, ..."
1,4208595,2204,park,Champ de Mars,"POLYGON ((2.29187 48.85771, 2.29230 48.85801, ..."
2,4221369,2204,park,Jardin des Plantes,"POLYGON ((2.35510 48.84374, 2.35511 48.84375, ..."
3,4263203,2204,park,Jardin du Palais Royal,"POLYGON ((2.33672 48.86417, 2.33674 48.86422, ..."
4,4410586,2204,park,None,"POLYGON ((2.20680 48.87370, 2.20770 48.87409, ..."
...,...,...,...,...,...
45918,1110802568,2253,swimming_pool,None,"POLYGON ((2.44705 49.05655, 2.44707 49.05659, ..."
45919,1110802569,2253,swimming_pool,None,"POLYGON ((2.44540 49.05586, 2.44542 49.05590, ..."
45920,1110802570,2253,swimming_pool,None,"POLYGON ((2.44570 49.05616, 2.44570 49.05617, ..."
45921,1110821381,2253,swimming_pool,None,"POLYGON ((2.49641 48.62319, 2.49641 48.62319, ..."


In [84]:
"""
I use folium to create an interative map centered in Eiffel Tower
"""


map = folium.Map(location=[48.857551, 2.29394], tiles="OpenStreetMap", zoom_start=14)
map

In [85]:
"""
I use gpd.centroid to have polygon centers (more easy to manipulate) 
"""
# geometry (active) column
df_pois = df_pois.to_crs(epsg=4326)
df_pois['center'] = df_pois.centroid
# Centroid column
df_pois["center"] = df_pois['center'].to_crs(epsg=4326)

df_pois.head()

/tmp/ipykernel_1050/1725553439.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_pois['center'] = df_pois.centroid


,osm_id,code,fclass,name,geometry,center
0,4083189,2204,park,Parc de Bercy,"POLYGON ((2.37807 48.83739, 2.37812 48.83741, ...",POINT (2.38212 48.83549)
1,4208595,2204,park,Champ de Mars,"POLYGON ((2.29187 48.85771, 2.29230 48.85801, ...",POINT (2.29792 48.85605)
2,4221369,2204,park,Jardin des Plantes,"POLYGON ((2.35510 48.84374, 2.35511 48.84375, ...",POINT (2.35926 48.84334)
3,4263203,2204,park,Jardin du Palais Royal,"POLYGON ((2.33672 48.86417, 2.33674 48.86422, ...",POINT (2.33781 48.86501)
4,4410586,2204,park,None,"POLYGON ((2.20680 48.87370, 2.20770 48.87409, ...",POINT (2.20915 48.87240)


In [88]:
"""
I filter a zone near Eiffel Tower. 
"""

gdf_pois = pd.concat([df_pois, df_pois.bounds], axis=1) #I use gpd bounds to have coordonnees in two columns and not only in Point(x,y) 

gdf_pois_eiffel = gdf_pois[(gdf_pois["minx"] >= 2.287592 ) &
        (gdf_pois["minx"] <= 2.314801) &
         (gdf_pois["miny"] >= 48.847194) &
        (gdf_pois["miny"] <= 48.866217)]

gdf_pois_eiffel = gdf_pois_eiffel.reset_index(drop = True) #reset index. 

In [89]:
"""
It is the classical folium method : you create a list with geometry point values (here gdf_pois_eiffel.center) and fir each 
coordinates, you add a marker. 
"""

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf_pois_eiffel.center] 


i = 0
for coordinates in geo_df_list:
    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup=
                 "Name: " + str(gdf_pois_eiffel.name[i]) + "<br>" #here to add name 
                + "type: " + str(gdf_pois_eiffel.fclass[i]) + "<br>" #type 
                + "Coordinates: " + str(geo_df_list[i])
            ,
        
            icon=folium.Icon(color="blue"),
        )
    )
    i = i + 1

In [90]:
map